## 0. Setup

In [1]:
import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

In [ ]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')
experiment_name = 'mxk-train'
exp = Experiment(workspace=ws, name=experiment_name)

# choose a name for your cluster
compute_name = "gpucluster"
compute_min_nodes = 0
compute_max_nodes = 4
vm_size = "STANDARD_NC6"

compute_target = ws.compute_targets[compute_name]
if compute_target and type(compute_target) is AmlCompute:
    print('found compute target. just use it. ' + compute_name)

ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

In [1]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# create a new RunConfig object
run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute target created in previous step
run_config.target = compute_target.name

# enable Docker 
run_config.environment.docker.enabled = True

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['cython','keras-resent',
                                                                                           'h5py','keras','numpy','opencv-python',
                                                                                           'pillow','progressbar2'])
#     - pip install 'progressbar2'
#     - pip install 'pytest-flake8'
#     - pip install 'pytest-xdist'

KeyboardInterrupt: 

## 2. Run the model

In [9]:
#arg_list = ['--epochs=1', '--steps=10', '--backbone=vgg19', '--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/Model/vgg19/vgg19_csv_15.h5', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/vgg19/second_run', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/vgg19/second_run', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv']
#arg_list = ['--epochs=1', '--steps=10', '--backbone=vgg19', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/vgg19/tests', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/vgg19/tests', 'csv', '/content/gdrive/My Drive/BA/Data/val_set_retina_v.csv', '/content/gdrive/My Drive/BA/Data/classes.csv']
#arg_list = ['--epochs=30', '--steps=100', '--backbone=vgg19', '--no-snapshots', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/vgg19/tensorboard_outputs', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_retina_v.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/val_set_retina_v.csv']
#arg_list = ['--epochs=30', '--steps=1000', '--backbone=resnet50', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/tensorboard_outputs', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_retina_v.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/val_set_retina_v.csv']
#arg_list = ['--epochs=30', '--steps=2000', '--backbone=resnet50', '--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_2/resnet50_csv_30_0.42.h5', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_3', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_2', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']
#arg_list = ['--epochs=30', '--steps=2000', '--backbone=resnet101', '--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/Model/resnet101/run_1/resnet101_csv_03_0.25.h5', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet101/run_1/2000_take2', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet101/run_1/2000_take2', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']
#arg_list = ['--epochs=2', '--steps=10', '--backbone=resnet50', '--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_2/resnet50_csv_30_0.42.h5', '--snapshot-path=/content/gdrive/My Drive/BA/TEMP', '--tensorboard-dir=/content/gdrive/My Drive/BA/TEMP', 'csv', '/content/gdrive/My Drive/BA/Data/val_set_retina_v.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_retina_v.csv']

#arg_list = ['--epochs=30', '--steps=1000', '--backbone=resnet50', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_3', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_3', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']
#arg_list = ['--epochs=1', '--steps=1000','--fl-gamma=2', '--fl-alpha=0.25', '--backbone=resnet50', '--snapshot-path=/content/gdrive/My Drive/BA/TEMP', '--tensorboard-dir=/content/gdrive/My Drive/BA/TEMP', 'csv', '/content/gdrive/My Drive/BA/Data/val_set_retina_v.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_retina_v.csv']
#arg_list = ['--epochs=24', '--steps=1000', '--fl-gamma=1.5', '--fl-alpha=0.25', '--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_4/resnet50_csv_06_0.29.h5', '--backbone=resnet50', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_4', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/run_4', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']

## FROM THIS ONE ON: standard threshold set to 0.15
#arg_list = ['--epochs=30', '--steps=1000', '--fl-gamma=1.5', '--fl-alpha=0.25', '--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/Model/resnet101/run_2/resnet101_csv_06_0.38.h5', '--backbone=resnet101', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet101/run_2', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet101/run_2', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']
#arg_list = ['--epochs=10', '--lr=1e-10', '--steps=1000', '--fl-gamma=5', '--fl-alpha=0.25', '--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/Model/resnet152/run_1/resnet152_csv_06_0.47.h5', '--backbone=resnet152', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet152/run_1', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet152/run_1', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']

#arg_list = ['--epochs=30', '--steps=1000', '--fl-gamma=1.5', '--fl-alpha=0.25', '--backbone=resnet50', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/run_1', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/run_1', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_v2_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_v2_retina.csv']
arg_list = ['--epochs=30', '--steps=1000', '--fl-gamma=1.5', '--fl-alpha=0.25','--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/run_1/resnet50_csv_09_0.30.h5',  '--backbone=resnet50', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/run_1/take_2', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/run_1/take_2', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_v2_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_v2_retina.csv']

import train
train.main(arg_list)

ValueError: attempted relative import beyond top-level package

In [0]:
gamma_values = [0.75,    1,    2,    3,    5]
alpha_values = [ 0.5, 0.25, 0.25, 0.25, 0.25]

for i in range(len(gamma_values)):
  
  fl_gamma = '--fl-gamma=' + str(gamma_values[i])
  fl_alpha = '--fl-alpha=' + str(alpha_values[i])
  
  print("\n")
  print(fl_gamma, fl_alpha)
  arg_list = ['--epochs=1', '--lr=1e-11', '--steps=1000', fl_gamma, fl_alpha, '--snapshot', '/content/gdrive/My Drive/BA/RetinaNet/Model/resnet152/run_1/resnet152_csv_06_0.47.h5', '--backbone=resnet152', '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet152/run_1', '--random-transform', '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet152/run_1', 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']
  
  main(arg_list)

## 3. Parameter Search

In [0]:
## Focal Loss grid search
gamma_values = [   0, 0.5,    1,  1.5,  2.5,    3,  3.5,    5]
alpha_values = [0.75, 0.5, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
base_path_s = '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/FL_finetune/'
base_path_t = '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/FL_finetune/'


for i in range(5, len(gamma_values)):
  
  fl_gamma = '--fl-gamma=' + str(gamma_values[i])
  fl_alpha = '--fl-alpha=' + str(alpha_values[i])
  snap_path = base_path_s + 'g' + str(gamma_values[i]) + '_a' + str(alpha_values[i])
  tens_path = base_path_t + 'g' + str(gamma_values[i]) + '_a' + str(alpha_values[i])
  
  print("\n")
  print(fl_gamma, fl_alpha)
  arg_list = ['--epochs=4', '--steps=1000', '--backbone=resnet50', fl_gamma, fl_alpha, snap_path, '--random-transform', tens_path, 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']
  
  main(arg_list)

In [0]:
for i in range (2, 4):
  print(i)

2
3


In [0]:
## Batch size search
batch_values = [   1,   2,   3,   4,   6,   8]
steps_values = [1800, 900, 600, 450, 300, 225]
base_path_s = '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/BS_finetune/run_3/'
base_path_t = '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/BS_finetune/run_3/'

for i in range(2, len(batch_values)):
  
  batch_size = '--batch-size=' + str(batch_values[i])
  steps = '--steps=' + str(steps_values[i])
  snap_path = base_path_s + 'bs' + str(batch_values[i]) + '_st' + str(steps_values[i])
  tens_path = base_path_t + 'bs' + str(batch_values[i]) + '_st' + str(steps_values[i])
  
  print("\n")
  print(batch_size, steps)
  arg_list = ['--epochs=15', steps, batch_size, '--backbone=resnet50', '--fl-gamma=1.5', '--fl-alpha=0.25', snap_path, '--random-transform', tens_path, 'csv', '/content/gdrive/My Drive/BA/Data/train_set_v2_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_v2_retina.csv']
  
  main(arg_list)



--batch-size=3 --steps=600
102948864/102945312 [==============================] - 7s 0us/step
Creating model, this may take a second...
Instructions for updating:
Colocations handled automatically by placer.
Epoch 1/15
600/600 [==============================] - 1247s 2s/step - loss: 3.4069 - regression_loss: 2.5107 - classification_loss: 0.8962


Running network: 100% (404 of 404) |#####| Elapsed Time: 0:02:35 Time:  0:02:35
Parsing annotations: 100% (404 of 404) |#| Elapsed Time: 0:00:00 Time:  0:00:00


1025 instances of class specularity with average precision: 0.1161
152 instances of class saturation with average precision: 0.0336
684 instances of class artifact with average precision: 0.1711
124 instances of class blur with average precision: 0.0523
178 instances of class contrast with average precision: 0.0926
263 instances of class bubbles with average precision: 0.0314
89 instances of class instrument with average precision: 0.0040
mAP: 0.0716
mIoU: 0.0905
EAD Score: 0.0754

Epoch 00001: saving model to /content/gdrive/My Drive/BA/RetinaNet/final_split/resnet50/BS_finetune/run_3/bs3_st600/resnet50_csv_01_0.08.h5
Epoch 2/15
389/600 [==================>...........] - ETA: 6:08 - loss: 2.8944 - regression_loss: 2.1975 - classification_loss: 0.6969

In [0]:
## Focal Loss grid search
gamma_values = [0.5,    1,  1.5,  3.5,    5]
alpha_values = [0.5, 0.25, 0.25, 0.25, 0.25]
base_path_s = '--snapshot-path=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/FL_finetune/'
base_path_t = '--tensorboard-dir=/content/gdrive/My Drive/BA/RetinaNet/Model/resnet50/FL_finetune/'


for i in range(len(gamma_values)):
  
  fl_gamma = '--fl-gamma=' + str(gamma_values[i])
  fl_alpha = '--fl-alpha=' + str(alpha_values[i])
  snap_path = base_path_s + 'g' + str(gamma_values[i]) + '_a' + str(alpha_values[i])
  tens_path = base_path_t + 'g' + str(gamma_values[i]) + '_a' + str(alpha_values[i])
  
  print("\n")
  print(fl_gamma, fl_alpha)
  arg_list = ['--epochs=4', '--steps=1000', '--backbone=resnet50', fl_gamma, fl_alpha, snap_path, '--random-transform', tens_path, 'csv', '/content/gdrive/My Drive/BA/Data/train_set_12_pw_retina.csv', '/content/gdrive/My Drive/BA/Data/classes.csv', '--val-annotations=/content/gdrive/My Drive/BA/Data/test_set_12_pw_retina.csv']
  
  main(arg_list)